# Census Income - Proyect with Random Forest (Modelo 6)

> Aqui vamos a equiparar a la clase minoritaria sobremuestrandola, para que al entrenar el modelo con una cantidad de clases relativamente equiparables

> Recordar que al aumentar datos de las clase minoritaria se esta generando ruido que evite que se identifique correctamente la clase mayoritaria, sin embargo en este caso no vamos a perder informacion.

Variables del conjunto de datos "Census Income" de UCI. Este conjunto de datos se utiliza comúnmente para tareas de clasificación, como predecir si una persona gana más o menos de $50,000 al año, en base a sus características. Las caracteristicas son:

1. **age** (edad): La edad de la persona en años.
2. **workclass** (tipo de trabajo): Clasificación del tipo de empleo (e.g., asalariado, autónomo, gobierno).
3. **fnlwgt** (ponderación final): Peso final calculado para las muestras en la encuesta, representa el número de personas que una fila puede representar.
4. **education** (educación): Nivel educativo alcanzado (e.g., escuela secundaria, licenciatura).
5. **education-num** (número de años de educación): Número de años de educación recibidos.
6. **marital-status** (estado civil): Estado civil de la persona (e.g., casado, soltero).
7. **occupation** (ocupación): Tipo de ocupación o profesión de la persona.
8. **relationship** (relación): Relación de la persona con el jefe de familia (e.g., esposo, esposa, hijo).
9. **race** (raza): Clasificación racial de la persona (e.g., blanca, negra, asiática).
10. **sex** (sexo): Sexo de la persona (masculino o femenino).
11. **capital-gain** (ganancia de capital): Ganancias de capital obtenidas por inversiones.
12. **capital-loss** (pérdida de capital): Pérdidas de capital incurridas.
13. **hours-per-week** (horas por semana): Número de horas trabajadas por semana.
14. **native-country** (país de origen): País de nacimiento de la persona.
15. **income** (ingreso): Ingreso anual, clasificado en ">50K" o "<=50K", indicando si la persona gana más o menos de $50,000 al año.



In [1]:
import pandas as pd

# Cargar el archivo de datos principal 'adult.data'
data_path = './adult.data'
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 
                'marital_status', 'occupation', 'relationship', 'race', 
                'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 
                'native_country', 'income']

# Cargar el dataset en un DataFrame
df = pd.read_csv(data_path, header=None, names=column_names, engine='python')

# Mostrar las primeras filas
df.head(2)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


In [2]:
from sklearn.preprocessing import OrdinalEncoder
X = df.drop(['income','native_country','race'], axis=1)
y = df['income']

ord_enc = OrdinalEncoder()
X = pd.DataFrame(ord_enc.fit_transform(X), columns=X.columns)

En este caso vamos a sobremuestrear a la clase minoritaria, para no perder informacion de parte de la clase mayoritaria como anteriormente se procedio, sim embargo corremos el riesgo de agregar ruido por las nuevas muestras que se generaran

In [3]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)


> Verificamos el balance de dlas clases 

In [7]:
from tools_ml import recursos_graficos as rg  

df_nex = pd.DataFrame({'income': y_resampled})

rg.histogram(df_nex, 'income')

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=69)

rf = RandomForestClassifier(bootstrap=True,
                            max_depth=20,
                            max_features= 5,
                            min_samples_leaf = 6,
                            min_samples_split= 2,
                            n_estimators= 200,
                            # class_weight = {' <=50K': 0.7, ' >50K': 1},

    random_state=42)
rf
# entrenamos el modelo
rf.fit(x_train, y_train)
rf

RandomForestClassifier(max_depth=20, max_features=5, min_samples_leaf=6,
                       n_estimators=200, random_state=42)

In [9]:
from sklearn.metrics import classification_report
y_pred = rf.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       <=50K       0.91      0.83      0.87      7362
        >50K       0.84      0.92      0.88      7470

    accuracy                           0.87     14832
   macro avg       0.88      0.87      0.87     14832
weighted avg       0.88      0.87      0.87     14832



In [10]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[6095, 1267],
       [ 618, 6852]], dtype=int64)

In [11]:
y_pred_origin = rf.predict(X)

print(classification_report(y, y_pred_origin))

              precision    recall  f1-score   support

       <=50K       0.97      0.86      0.91     24720
        >50K       0.68      0.92      0.78      7841

    accuracy                           0.88     32561
   macro avg       0.82      0.89      0.85     32561
weighted avg       0.90      0.88      0.88     32561



In [12]:
confusion_matrix(y,y_pred_origin)

array([[21266,  3454],
       [  611,  7230]], dtype=int64)